# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,5.53,96,100,6.73,GS,1712506152
1,1,port mathurin,-19.6833,63.4167,27.24,79,45,6.04,MU,1712506152
2,2,yellowknife,62.4560,-114.3525,1.34,91,75,7.20,CA,1712506016
3,3,beloyarskiy,63.7119,66.6722,-3.33,75,100,5.81,RU,1712506153
4,4,port-aux-francais,-49.3500,70.2167,5.24,69,100,6.04,TF,1712506153
...,...,...,...,...,...,...,...,...,...,...
563,563,kharp,66.8014,65.8081,-4.60,95,100,4.51,RU,1712506224
564,564,valparaiso,-33.0393,-71.6273,25.08,60,0,4.12,CL,1712506224
565,565,vorkuta,67.5000,64.0000,-7.16,87,100,6.71,RU,1712506013
566,566,santa maria del rio,21.8000,-100.7500,25.03,21,38,3.42,MX,1712506224


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:

map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700, #optional
    frame_height = 500 #optional
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
filtered_cities = city_data_df[city_data_df['Cloudiness'] == 0]
filtered_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,brak,27.5496,14.2714,29.81,6,0,3.79,LY,1712506153
14,14,la chapelle-saint-mesmin,47.8865,1.8345,19.47,68,0,4.12,FR,1712506154
29,29,al bardiyah,31.7561,25.0865,18.55,59,0,9.00,LY,1712506156
47,47,anadyr,64.7500,177.4833,-20.56,83,0,5.00,RU,1712506158
55,55,fort bragg,35.1390,-79.0060,17.12,40,0,2.57,US,1712505920
...,...,...,...,...,...,...,...,...,...,...
544,544,broome,42.2506,-75.8330,11.21,49,0,5.14,US,1712506221
550,550,nica,43.7031,7.2661,20.53,70,0,3.09,FR,1712506222
551,551,misratah,32.3754,15.0925,21.73,39,0,3.88,LY,1712506222
560,560,khairpur mir's,27.5295,68.7617,29.00,10,0,0.94,PK,1712506223


In [37]:
null_counts = filtered_cities.isnull().sum()
print(null_counts)

City_ID       0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       1
Date          0
dtype: int64


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,grytviken,GS,-54.2811,-36.5092,96,
1,port mathurin,MU,-19.6833,63.4167,79,
2,yellowknife,CA,62.4560,-114.3525,91,
3,beloyarskiy,RU,63.7119,66.6722,75,
4,port-aux-francais,TF,-49.3500,70.2167,69,
...,...,...,...,...,...,...
563,kharp,RU,66.8014,65.8081,95,
564,valparaiso,CL,-33.0393,-71.6273,60,
565,vorkuta,RU,67.5000,64.0000,87,
566,santa maria del rio,MX,21.8000,-100.7500,21,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
#Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

grytviken - nearest hotel: No hotel found
port mathurin - nearest hotel: Escale Vacances
yellowknife - nearest hotel: No hotel found
beloyarskiy - nearest hotel: Сибирь
port-aux-francais - nearest hotel: Keravel
talcahuano - nearest hotel: Club Recreativo Armada de Chile
port douglas - nearest hotel: Mantra Aqueous Port Douglas
moss bluff - nearest hotel: Comfort Suites Lake Charles
brak - nearest hotel: فندق براك السياحي
avarua - nearest hotel: Paradise Inn
cacheu - nearest hotel: Delta Hotel
talnakh - nearest hotel: Талнах
adamstown - nearest hotel: No hotel found
margate - nearest hotel: The Reading Rooms
la chapelle-saint-mesmin - nearest hotel: Formule 1
westport - nearest hotel: Norwalk Inn and Conference Center
papatowai - nearest hotel: No hotel found
kokopo - nearest hotel: Hotel Kokopo
nago - nearest hotel: グリーンリッチホテル
pariaman - nearest hotel: HOTEL NAN TONGGA
whitehorse - nearest hotel: Town & Mountain Hotel
cabo san lucas - nearest hotel: Comfort Rooms
tolanaro - nearest ho

,City,Country,Lat,Lng,Humidity,Hotel Name
0,grytviken,GS,-54.2811,-36.5092,96,No hotel found
1,port mathurin,MU,-19.6833,63.4167,79,Escale Vacances
2,yellowknife,CA,62.4560,-114.3525,91,No hotel found
3,beloyarskiy,RU,63.7119,66.6722,75,Сибирь
4,port-aux-francais,TF,-49.3500,70.2167,69,Keravel
...,...,...,...,...,...,...
563,kharp,RU,66.8014,65.8081,95,Собь
564,valparaiso,CL,-33.0393,-71.6273,60,Ibis Hotel
565,vorkuta,RU,67.5000,64.0000,87,"""Мегаполис"""
566,santa maria del rio,MX,21.8000,-100.7500,21,Hotel del Rio


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
# Filter hotel_df to exclude rows where "Hotel Name" is "No hotel found"
valid_hotels_df = hotel_df[hotel_df['Hotel Name'] != 'No hotel found']

# Plot the valid hotels using hvplot
plot = valid_hotels_df.hvplot.points(x='Lng', y='Lat', geo=True, color='City', size=100, alpha=0.6, tiles = 'OSM', hover_cols=['City', 'Hotel Name'])

# Show the plot
plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name)

In [ ]:

map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700, #optional
    frame_height = 500 #optional
)

# Display the map
map_plot

In [24]:
valid_hotels_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,port mathurin,MU,-19.6833,63.4167,79,Escale Vacances
3,beloyarskiy,RU,63.7119,66.6722,75,Сибирь
4,port-aux-francais,TF,-49.3500,70.2167,69,Keravel
5,talcahuano,CL,-36.7167,-73.1167,69,Club Recreativo Armada de Chile
6,port douglas,AU,-16.4833,145.4667,76,Mantra Aqueous Port Douglas
...,...,...,...,...,...,...
563,kharp,RU,66.8014,65.8081,95,Собь
564,valparaiso,CL,-33.0393,-71.6273,60,Ibis Hotel
565,vorkuta,RU,67.5000,64.0000,87,"""Мегаполис"""
566,santa maria del rio,MX,21.8000,-100.7500,21,Hotel del Rio
